In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')
import plotly
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
def labelEncode(df, colName):
    lb_make = LabelEncoder()
    df[colName + "_int"] = lb_make.fit_transform(df[colName])
    df.drop(colName, inplace=True, axis=1)
    # df[[colName, colName + "_int"]].head()

# Postulantes/Educacion

In [3]:
# Asigno puntajes a los diferentes niveles educativos encontrados para poder 
# obtener asi la educacion mas valiosa (la de nivel academico mas alto) de cada postulante.
valores_educacion = pd.DataFrame({
    'valor_educacion': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    'nombre': [
        'Otro',  'Otro',  'Otro',  'Secundario',  'Secundario',  'Secundario',  'Terciario/Técnico', 'Universitario',  \
        'Terciario/Técnico', 'Universitario',  'Terciario/Técnico', 'Universitario',  'Posgrado', 'Master','Posgrado', \
        'Master','Posgrado', 'Master','Doctorado','Doctorado','Doctorado'
    ],
    'estado': [
        'Abandonado', 'En Curso','Graduado','Abandonado', 'En Curso','Graduado','Abandonado', 'Abandonado', 'En Curso',\
        'En Curso','Graduado','Graduado','Abandonado', 'Abandonado', 'En Curso','En Curso','Graduado','Graduado',\
        'Abandonado', 'En Curso','Graduado'
    ]
})
valores_educacion.to_csv('../data/adri_data/valor_educacion.csv', sep=',', encoding='utf-8', index=False)
# valores_educacion

In [4]:
post_educacion_1  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion.csv')
post_educacion_2  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion_2.csv')
post_educacion_3  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion_3.csv')

post_educacion = pd.concat([post_educacion_1, post_educacion_2, post_educacion_3], axis=0)
post_educacion.count()              # 1103193

idpostulante    1103193
nombre          1103193
estado          1103193
dtype: int64

In [5]:
# post_educacion.drop_duplicates(inplace = True)

# Hago el join para aplicar puntajes a las educaciones de los postulantes.
post_educacion = pd.merge(post_educacion, valores_educacion, how='inner', \
                         left_on=['nombre','estado'], right_on = ['nombre','estado'])

# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_educacion.groupby(['idpostulante'])['valor_educacion'].max().reset_index()
max_post_educacion.count() # 447909

idpostulante       447909
valor_educacion    447909
dtype: int64

In [6]:
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_educacion = pd.merge(post_educacion, max_post_educacion,  how='inner', \
                         left_on=['idpostulante','valor_educacion'], \
                         right_on = ['idpostulante','valor_educacion'])


In [7]:
post_educacion['educacion'] = post_educacion['nombre'] + ' ' + post_educacion['estado'] 
post_educacion.head()

,idpostulante,nombre,estado,valor_educacion,educacion
0,8rYD,Master,En Curso,16,Master En Curso
1,8rYD,Master,En Curso,16,Master En Curso
2,e4VXj,Master,En Curso,16,Master En Curso
3,a6GvZN,Master,En Curso,16,Master En Curso
4,a6GvZN,Master,En Curso,16,Master En Curso


In [8]:
labelEncode(post_educacion, 'educacion')
labelEncode(post_educacion, 'nombre')
labelEncode(post_educacion, 'estado')
post_educacion.head()

,idpostulante,valor_educacion,educacion_int,nombre_int,estado_int
0,8rYD,16,4,1,1
1,8rYD,16,4,1,1
2,e4VXj,16,4,1,1
3,a6GvZN,16,4,1,1
4,a6GvZN,16,4,1,1


In [9]:
# post_educacion.drop(['nombre', 'estado', 'educacion'], inplace = True, axis=1)
post_educacion.drop_duplicates(inplace=True)

In [10]:
post_educacion.to_csv('../data/adri_data/postulantes_educacion.csv', sep=',', encoding='utf-8', index=False)
post_educacion.count()

idpostulante       447909
valor_educacion    447909
educacion_int      447909
nombre_int         447909
estado_int         447909
dtype: int64

In [16]:
post_educacion.head()

,idpostulante,valor_educacion,educacion_int
0,8rYD,16,4
2,e4VXj,16,4
3,a6GvZN,16,4
5,5264LM,16,4
7,5MlbrN,16,4


# Postulantes/Genero/Edad

In [11]:
post_gen_edad_1 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad.csv')
post_gen_edad_2 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad_2.csv')
post_gen_edad_3 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad_3.csv')

post_gen_edad = pd.concat([post_gen_edad_1, post_gen_edad_2, post_gen_edad_3], axis=0)
post_gen_edad.count()

idpostulante       780020
fechanacimiento    745747
sexo               780020
dtype: int64

In [12]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].astype('category')
post_gen_edad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780020 entries, 0 to 200887
Data columns (total 3 columns):
idpostulante       780020 non-null object
fechanacimiento    745747 non-null object
sexo               780020 non-null category
dtypes: category(1), object(2)
memory usage: 18.6+ MB


In [13]:
post_gen_edad.drop_duplicates(inplace=True)
post_gen_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [14]:
post_gen_edad.loc[post_gen_edad['sexo'] == '0.0','sexo'] = np.nan
post_gen_edad.loc[post_gen_edad['sexo'] == 'NO_DECLARA','sexo'] = np.nan

cols = post_gen_edad.select_dtypes(['category']).columns
post_gen_edad[cols] = post_gen_edad[cols].apply(lambda x: x.cat.codes)

post_gen_edad.count() 

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [15]:
# Modifico el tipo de dato de la fecha.
post_gen_edad['fechanacimiento'] = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad['edad'] = (now - post_gen_edad['fechanacimiento']).astype('<m8[Y]')

In [17]:
post_gen_edad['birth_year'] = post_gen_edad['fechanacimiento'].dt.year

In [18]:
post_gen_edad['birth_month'] = post_gen_edad['fechanacimiento'].dt.month

In [19]:
post_gen_edad['birth_day'] = post_gen_edad['fechanacimiento'].dt.day

In [21]:
post_gen_edad.to_csv('../data/adri_data/postulantes_genero_edad.csv', sep=',', encoding='utf-8', index=False)

In [22]:
post_gen_edad = pd.read_csv('../data/adri_data/postulantes_genero_edad.csv')
post_gen_edad.drop_duplicates('idpostulante', inplace=True) 
post_gen_edad.count() # 504407 
# post_gen_edad.count() # 505382 

idpostulante       504407
fechanacimiento    477883
sexo               504407
edad               477883
birth_year         477883
birth_month        477883
birth_day          477883
dtype: int64

In [23]:
post_gen_edad.fillna(0, inplace=True)
post_gen_edad['edad'] = post_gen_edad['edad'].astype(int)
post_gen_edad['birth_year'] = post_gen_edad['birth_year'].astype(int)
post_gen_edad['birth_month'] = post_gen_edad['birth_month'].astype(int)
post_gen_edad['birth_day'] = post_gen_edad['birth_day'].astype(int)

In [24]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].apply(lambda x: 0 if x == -1 else (-1 if x == 2 else 1))

In [26]:
post_gen_edad.drop('fechanacimiento', inplace=True, axis=1)

In [27]:
post_gen_edad.head()

,idpostulante,sexo,edad,birth_year,birth_month,birth_day
0,6MM,-1,33,1985,1,1
1,Nzz,0,0,0,0,0
2,ZX1,0,0,0,0,0
3,Nq5,0,0,0,0,0
4,ebE,-1,65,1952,7,7


In [28]:
post_gen_edad.to_csv('../data/adri_data/postulantes_genero_edad.csv', sep=',', encoding='utf-8', index=False)

# Merge de Postulantes

In [30]:
postulantes = pd.merge(post_educacion, post_gen_edad, how='outer', on='idpostulante')
postulantes.shape

(504407, 10)

In [31]:
postulantes.head()

,idpostulante,valor_educacion,educacion_int,nombre_int,estado_int,sexo,edad,birth_year,birth_month,birth_day
0,8rYD,16.0,4.0,1.0,1.0,-1,43,1974,12,11
1,e4VXj,16.0,4.0,1.0,1.0,-1,50,1968,4,25
2,a6GvZN,16.0,4.0,1.0,1.0,-1,48,1970,4,14
3,5264LM,16.0,4.0,1.0,1.0,-1,41,1976,7,30
4,5MlbrN,16.0,4.0,1.0,1.0,-1,40,1978,1,8


In [32]:
postulantes.fillna(-1, inplace=True)

In [33]:
postulantes.to_csv('../data/adri_data/postulantes.csv', sep=',', encoding='utf-8', index=False)

In [34]:
postulantes = pd.read_csv('../data/adri_data/postulantes.csv', sep=',', encoding='utf-8')
postulantes.head()

,idpostulante,valor_educacion,educacion_int,nombre_int,estado_int,sexo,edad,birth_year,birth_month,birth_day
0,8rYD,16.0,4.0,1.0,1.0,-1,43,1974,12,11
1,e4VXj,16.0,4.0,1.0,1.0,-1,50,1968,4,25
2,a6GvZN,16.0,4.0,1.0,1.0,-1,48,1970,4,14
3,5264LM,16.0,4.0,1.0,1.0,-1,41,1976,7,30
4,5MlbrN,16.0,4.0,1.0,1.0,-1,40,1978,1,8


# Vistas

In [32]:
vistas_1 = pd.read_csv('../data/tp2_data/fiuba_3_vistas.csv')
vistas_2 = pd.read_csv('../data/tp2_data/fiuba_3_vistas_2.csv')
vistas_3 = pd.read_csv('../data/tp2_data/fiuba_3_vistas_3.csv')

vistas = pd.concat([vistas_1, vistas_2, vistas_3], axis=0)
vistas.drop_duplicates(inplace=True)
vistas.rename({'idAviso' : 'idaviso'}, inplace =True, axis=1)

In [33]:
vistas['timestamp'] = vistas['timestamp'].apply(pd.to_datetime)

KeyboardInterrupt: 

In [ ]:
now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)

vistas['timestamp'] = vistas['timestamp'].apply(lambda x: x.total_seconds()).astype(int) 

vistas['month'] = vistas['timestamp'].apply(lambda x: x.month).astype(int) 
vistas['day'] = vistas['timestamp'].apply(lambda x: x.day).astype(int) 
vistas['dayofweek'] = vistas['timestamp'].apply(lambda x: x.dayofweek).astype(int) 
vistas['hour'] = vistas['timestamp'].apply(lambda x: x.hour).astype(int) 

In [ ]:
vistas.drop('timestamp', axis=1, inplace=True)
vistas.head()

In [ ]:
vistas.to_csv('../data/adri_data/vistas.csv', sep=',', encoding='utf-8', index=False)

# Postulaciones

In [ ]:
postulaciones_1 = pd.read_csv('../data/tp2_data/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('../data/tp2_data/fiuba_4_postulaciones_2.csv')
postulaciones = pd.concat([postulaciones_1, postulaciones_2], axis=0)

postulaciones.drop_duplicates(inplace=True)

In [ ]:
postulaciones.rename({'fechapostulacion': 'timestamp'}, axis=1, inplace=True)
postulaciones.head()

In [ ]:
postulaciones['timestamp'] = postulaciones['timestamp'].apply(pd.to_datetime)

In [ ]:
# now = pd.Timestamp(dt.datetime.now())
# pd.to_datetime(now)

# postulaciones['timestamp'] = (now - postulaciones['timestamp']).apply(lambda x: x.total_seconds())
# postulaciones['year'] = postulaciones['timestamp'].apply(lambda x: x.year).astype(int) 
postulaciones['month'] = postulaciones['timestamp'].apply(lambda x: x.month).astype(int) 
postulaciones['day'] = postulaciones['timestamp'].apply(lambda x: x.day).astype(int) 
postulaciones['dayofweek'] = postulaciones['timestamp'].apply(lambda x: x.dayofweek).astype(int) 
postulaciones['hour'] = postulaciones['timestamp'].apply(lambda x: x.hour).astype(int) 
postulaciones.drop('timestamp', axis=1, inplace=True)

In [ ]:
postulaciones.to_csv('../data/adri_data/postulaciones.csv', sep=',', encoding='utf-8', index=False)

postulaciones.head()

# Avisos Online

In [ ]:
avisos_online_1 = pd.read_csv('../data/tp2_data/fiuba_5_avisos_online.csv')
avisos_online_2 = pd.read_csv('../data/tp2_data/fiuba_5_avisos_online_2.csv')
avisos_online = pd.concat([avisos_online_1, avisos_online_2], axis=0)

avisos_online.drop_duplicates(inplace=True)
avisos_online.to_csv('../data/adri_data/avisos_online.csv', sep=',', encoding='utf-8', index=False)

avisos_online.head()

# Detalle Avisos

In [2]:
avisos_detalles_1 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_2.csv')
avisos_detalles_3 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_3.csv')
avisos_detalles_4 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_4.csv')
avisos_detalles = pd.concat([avisos_detalles_1, avisos_detalles_2, avisos_detalles_3, avisos_detalles_4], axis=0)

In [3]:
avisos_detalles.drop('idpais', axis=1, inplace=True)
avisos_detalles.drop_duplicates('idaviso', inplace=True)

In [35]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [5]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica',:].shape

(14, 10)

In [6]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica','nombre_area'] = 'Bioquimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica',:].shape

(0, 10)

In [7]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química',:].shape

(9, 10)

In [8]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química','nombre_area'] = 'Quimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química',:].shape

(0, 10)

In [9]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social',:].shape

(3, 10)

In [10]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social','nombre_area'] = 'Trabajo Social'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social',:].shape

(0, 10)

In [11]:
# areas = pd.DataFrame(avisos_detalles['nombre_area'].unique())
# areas.to_csv('../data/adri_data/areas_intermedio.csv', encoding='utf-8', index=False)
# areas

In [12]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [13]:
# avisos_detalles.duplicated().sum()

In [14]:
# avd = avisos_detalles[avisos_detalles.duplicated('idaviso')]['idaviso'].reset_index()
# avd = pd.merge(avd, avisos_detalles)
# avd.to_csv('../data/tp2_data/duplex.csv')

In [15]:
# zona = pd.DataFrame(avisos_detalles['nombre_zona'].unique())
# zona

In [41]:
avisos_detalles.to_csv('../data/adri_data/avisos_detalle_0.csv', sep=',', encoding='utf-8', index=False)

NameError: name 'avisos_detalles' is not defined

# Despues de la Unificacion

In [62]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalle_0.csv', encoding='utf-8')
avisos_online = pd.read_csv('../data/adri_data/avisos_online.csv', sep=',', encoding='utf-8')
avisos_detalles.shape

(25288, 10)

In [63]:
avisos_online['esta_online'] = 1

In [64]:
avisos_detalles = pd.merge(avisos_detalles, avisos_online, how='left', on='idaviso')

In [65]:
avisos_detalles['esta_online'] = avisos_detalles['esta_online'].apply(lambda x: 1 if x == 1 else -1)
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online
0,1111556097,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1
1,1111949392,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1
3,1112146010,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1
4,1112211475,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1


In [66]:
def cleanText(t):
    t = t.lower()\
        .replace('"', ' ') \
        .replace('*', ' ') \
        .replace('(', ' ') \
        .replace(')', ' ') \
        .replace('!', ' ') \
        .replace('¡', ' ') \
        .replace('¿', ' ') \
        .replace('?', ' ') \
        .replace('-', ' ') \
        .replace('/', ' ') \
        .replace('á', 'a') \
        .replace('é', 'e') \
        .replace('í', 'i') \
        .replace('ó', 'o') \
        .replace('ú', 'u') \
        .replace('ë', 'e') \
        .replace('ü', 'u') \
        .replace(',', ' ') \
        .replace('.', ' ') \
        .replace('-', ' ') \
        .replace('=', ' ') \
        .replace(':', ' ') \
        .replace(';', ' ') \
        .replace('&nbsp;', ' ') \
        .replace('&amp;', ' ') \
        .replace('&quot;', ' ') \
        .replace('&iquest;', ' ') \
        .replace('&lt;', ' ') 
    t = re.sub(r"(</?[\w\d\s]+>)", " ", t)
    t = t\
        .replace('<', ' ') \
        .replace('>', ' ') \
        .replace('\r', ' ') \
        .replace('\n', ' ')
        
    return t.strip()

def cleanCollection(s):
    s['palabra'] = s['palabra'].str.strip()
    s = s[s['palabra'] != '']
    s = s[s['palabra'] != 'de']
    s = s[s['palabra'] != 'para']
    s = s[s['palabra'] != 'y']
    s = s[s['palabra'] != 'a']
    s = s[s['palabra'] != 'la']
    s = s[s['palabra'] != 'en']
    s = s[s['palabra'] != 'el']
    s = s[s['palabra'] != 'con']
    s = s[s['palabra'] != 'zona']
    s = s[s['palabra'] != 'o']
    s = s[s['palabra'] != 'los']
    s = s[s['palabra'] != 'se']
    s = s[s['palabra'] != 'del']
    s = s[s['palabra'] != 'las']
    s = s[s['palabra'] != 'un']
    s = s[s['palabra'] != 'al']
    s = s[s['palabra'] != 'una']
    s = s[s['palabra'] != 'que']
    s = s[s['palabra'] != 'busqueda']
    s = s[s['palabra'] != 'por']
    s = s[s['palabra'] != 'años']
    s = s[s['palabra'] != 'es']
    s = s[s['palabra'] != '·']
    s = s[s['palabra'] != 'e']
    s = s[s['palabra'] != 'sus']
    s = s[s['palabra'] != 'no']
    s = s[s['palabra'] != 'and']
    s = s[s['palabra'] != 'su']
    s = s[s['palabra'] != 'como']
    s = s[s['palabra'] != 'area']
    s = s[s['palabra'] != 'nos']
    s = s[s['palabra'] != 'lunes']
    s = s[s['palabra'] != 'nuestro']
    s = s[s['palabra'] != 'poder']
    s = s[s['palabra'] != 'mismo']
    s = s[s['palabra'] != 'san']
    s = s[s['palabra'] != 'z']
    s = s[s['palabra'] != '']
    s = s[s['palabra'] != 'as']
    s = s[s['palabra'] != 'p']
    s = s[s['palabra'] != 'c']
    s = s[s['palabra'] != 'yr']
    s = s[s['palabra'] != '03']
    s = s[s['palabra'] != 'underline']
    s = s[s['palabra'] != 'decoration']
    s = s[s['palabra'] != 'entre']
    s = s[s['palabra'] != 'of']
    s = s[s['palabra'] != 'text']
    s = s[s['palabra'] != 'span']
    s = s[s['palabra'] != 'to']
    s = s[s['palabra'] != 'style']
    s = s[s['palabra'] != 'si']
    s = s[s['palabra'] != 'te']
    s = s[s['palabra'] != 'tu']
    s = s[s['palabra'] != 'hs']
    s = s[s['palabra'] != 'the']
    return s

In [67]:
avisos_detalles['titulo'] = avisos_detalles['titulo'].apply(lambda x:cleanText(x))
avisos_detalles['descripcion'] = avisos_detalles['descripcion'].apply(lambda x:cleanText(x))

In [68]:
avisos_detalles['descripcion'].head()

0    buscamos un  responsable de producto  para inc...
1    para nuestro cliente  concesionario chevrolet ...
2    importante estudio de auditoria y consultoria ...
3    chevron is a multinational company in the ener...
4    importante empresa de energias renovables  se ...
Name: descripcion, dtype: object

In [69]:
avisos_detalles['titulo'].head()

0    responsable de marketing producto div  constru...
1            asesor de servicios   postventa chevrolet
2                   analistas de impuestos sr ssr y jr
3                                    software engineer
4                           business developer manager
Name: titulo, dtype: object

### Detalle Avisos (Titulo)

In [70]:
avisos_detalles_titulo = pd.DataFrame(avisos_detalles['titulo'].unique(), columns=['titulo'])
avisos_detalles_titulo = avisos_detalles_titulo.sort_values('titulo').reset_index(drop=True)
avisos_detalles_titulo.head()

,titulo
0,
1,03 mar chief software architect usd$ 100k y...
2,03 mar cloud chief software architect usd$1...
3,03 mar group controller $200k year onlin...
4,03 mar inside sales representative usd$ 100...


In [71]:
avisos_detalles_descripcion = pd.DataFrame(avisos_detalles['descripcion'].unique(), columns=['descripcion'])
avisos_detalles_descripcion = avisos_detalles_descripcion.sort_values('descripcion').reset_index(drop=True)
avisos_detalles_descripcion.head()

,descripcion
0,
1,&iquest sos graduado o est&aacute s a punto de...
2,&lt p planeta zenok incorporara a su sta...
3,&lt p call center incorpora para ...
4,&lt p importante empresa de mayorista de tele...


In [72]:
joined_titulo = ' '.join(avisos_detalles_titulo['titulo'])
joined_descripcion = ' '.join(avisos_detalles_descripcion['descripcion'])
joined_all = joined_titulo + ' ' + joined_descripcion

In [73]:
splitted = joined_titulo.split(' ')
splitted = pd.DataFrame(splitted, columns=['palabra'])

splitted = pd.DataFrame(splitted.groupby('palabra')['palabra'].count())
splitted.rename({'palabra': 'count'}, inplace=True, axis=1)
splitted.index.name = 'palabra'
splitted = splitted.reset_index()
splitted.sort_values('count', ascending = False, inplace=True)

splitted['length'] = splitted['palabra'].apply(lambda x: len(x))

splitted = cleanCollection(splitted)

splitted = splitted.reset_index(drop=True)
splitted.drop(44, inplace=True)
splitted = splitted.head(63)
splitted.to_csv('../data/adri_data/palabras_titulo.csv', encoding = 'utf-8', index=False)
splitted

,palabra,count,length
0,analista,2231,8
1,tecnico,1158,7
2,administrativo,924,14
3,vendedor,886,8
4,comercial,821,9
5,ventas,758,6
6,sr,696,2
7,importante,676,10
8,jefe,603,4
9,empresa,583,7


In [74]:
def toBin(y):
    c = ''                                                             
    for x in splitted.palabra:
        c += '1' if x in y else '0'                                                              

    return int(c, 2)

In [75]:
avisos_detalles['titulo_2'] = avisos_detalles['titulo'].apply(lambda x : toBin(x))    

In [76]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_2
0,1111556097,responsable de marketing producto div constru...,buscamos un responsable de producto para inc...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640
1,1111949392,asesor de servicios postventa chevrolet,para nuestro cliente concesionario chevrolet ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952
2,1112145935,analistas de impuestos sr ssr y jr,importante estudio de auditoria y consultoria ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408
3,1112146010,software engineer,chevron is a multinational company in the ener...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0
4,1112211475,business developer manager,importante empresa de energias renovables se ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8


In [77]:
splitted = joined_descripcion.split(' ')
splitted = pd.DataFrame(splitted, columns=['palabra'])

splitted = pd.DataFrame(splitted.groupby('palabra')['palabra'].count())
splitted.rename({'palabra': 'count'}, inplace=True, axis=1)
splitted.index.name = 'palabra'
splitted = splitted.reset_index()
splitted.sort_values('count', ascending = False, inplace=True)

splitted['length'] = splitted['palabra'].apply(lambda x: len(x))

splitted = cleanCollection(splitted)
splitted = splitted.reset_index(drop=True)

splitted.drop(21, inplace=True)
splitted = splitted.head(63)
splitted.to_csv('../data/adri_data/palabras_descripcion.csv', encoding = 'utf-8', index=False)
# splitted = splitted[splitted['count'] > 8000]
splitted

,palabra,count,length
0,experiencia,27581,11
1,trabajo,18026,7
2,empresa,16000,7
3,requisitos,11387,10
4,excluyente,10725,10
5,tareas,10480,6
6,importante,10018,10
7,manejo,9940,6
8,equipo,9910,6
9,cliente,9013,7


In [78]:
avisos_detalles['descripcion_2'] = avisos_detalles['descripcion'].apply(lambda x : toBin(x))    
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_2,descripcion_2
0,1111556097,responsable de marketing producto div constru...,buscamos un responsable de producto para inc...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249
1,1111949392,asesor de servicios postventa chevrolet,para nuestro cliente concesionario chevrolet ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002
2,1112145935,analistas de impuestos sr ssr y jr,importante estudio de auditoria y consultoria ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304
3,1112146010,software engineer,chevron is a multinational company in the ener...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648
4,1112211475,business developer manager,importante empresa de energias renovables se ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8,5853804619654291768


In [79]:
splitted = joined_all.split(' ')
splitted = pd.DataFrame(splitted, columns=['palabra'])

splitted = pd.DataFrame(splitted.groupby('palabra')['palabra'].count())
splitted.rename({'palabra': 'count'}, inplace=True, axis=1)
splitted.index.name = 'palabra'
splitted = splitted.reset_index()
splitted.sort_values('count', ascending = False, inplace=True)

splitted['length'] = splitted['palabra'].apply(lambda x: len(x))

splitted = cleanCollection(splitted)

splitted = splitted.reset_index(drop=True)
splitted.drop(44, inplace=True)
splitted = splitted.head(63)
splitted.to_csv('../data/adri_data/palabras_titulo.csv', encoding = 'utf-8', index=False)
splitted

,palabra,count,length
0,experiencia,28072,11
1,trabajo,18059,7
2,empresa,16583,7
3,requisitos,11387,10
4,excluyente,10814,10
5,importante,10694,10
6,tareas,10516,6
7,manejo,10010,6
8,equipo,9937,6
9,cliente,9279,7


In [80]:
avisos_detalles['contenido'] = avisos_detalles['titulo'] + ' ' + avisos_detalles['descripcion']

In [81]:
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_2,descripcion_2,contenido
0,1111556097,responsable de marketing producto div constru...,buscamos un responsable de producto para inc...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249,responsable de marketing producto div constru...
1,1111949392,asesor de servicios postventa chevrolet,para nuestro cliente concesionario chevrolet ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002,asesor de servicios postventa chevrolet para...
2,1112145935,analistas de impuestos sr ssr y jr,importante estudio de auditoria y consultoria ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304,analistas de impuestos sr ssr y jr importante ...
3,1112146010,software engineer,chevron is a multinational company in the ener...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648,software engineer chevron is a multinational c...
4,1112211475,business developer manager,importante empresa de energias renovables se ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8,5853804619654291768,business developer manager importante empresa ...


In [82]:
avisos_detalles['contenido'] = avisos_detalles['contenido'].apply(lambda x : toBin(x))    
avisos_detalles.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo_2,descripcion_2,contenido
0,1111556097,responsable de marketing producto div constru...,buscamos un responsable de producto para inc...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249,8159438729029635585
1,1111949392,asesor de servicios postventa chevrolet,para nuestro cliente concesionario chevrolet ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002,7342170615448076428
2,1112145935,analistas de impuestos sr ssr y jr,importante estudio de auditoria y consultoria ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304,4791830003540049985
3,1112146010,software engineer,chevron is a multinational company in the ener...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648,2147483648
4,1112211475,business developer manager,importante empresa de energias renovables se ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8,5853804619654291768,5925316576974522051


In [83]:
# avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_titulo, how='inner', on='titulo')
avisos_detalles.drop(['titulo', 'descripcion'], inplace=True, axis=1)
avisos_detalles.rename({'titulo_2' : 'titulo', 'descripcion_2' : 'descripcion'}, inplace=True, axis=1)
avisos_detalles.count()

idaviso                 25288
nombre_zona             25288
ciudad                    160
mapacalle                2069
tipo_de_trabajo         25288
nivel_laboral           24953
nombre_area             25288
denominacion_empresa    25281
esta_online             25288
titulo                  25288
descripcion             25288
contenido               25288
dtype: int64

In [84]:
avisos_detalles.to_csv('../data/adri_data/avisos_detalle_1.csv', sep=',', encoding='utf-8', index=False)

# Despues de Calcular Contenido

In [35]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalle_1.csv', encoding='utf-8')
avisos_detalles.head()

,idaviso,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo,descripcion,contenido
0,1111556097,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249,8159438729029635585
1,1111949392,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002,7342170615448076428
2,1112145935,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304,4791830003540049985
3,1112146010,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648,2147483648
4,1112211475,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8,5853804619654291768,5925316576974522051


### Detalle Avisos (Nombre Zona)

In [147]:
avisos_detalles_zona = pd.DataFrame(avisos_detalles['nombre_zona'].unique(), columns=['nombre_zona'])
avisos_detalles_zona['idzona'] = avisos_detalles_zona['nombre_zona']
labelEncode(avisos_detalles_zona, 'idzona')
avisos_detalles_zona

,nombre_zona,idzona_int
0,Gran Buenos Aires,7
1,Capital Federal,1
2,GBA Oeste,6
3,Buenos Aires (fuera de GBA),0
4,La Plata,8
5,Santa Cruz,13
6,Tucuman,15
7,Cordoba,4
8,San Juan,12
9,Ciudad de Mendoza,3


In [148]:
avisos_detalles_zona.to_csv('../data/adri_data/avisos_detalle_zona.csv', sep=',', encoding='utf-8', index=False)
avisos_detalles_zona = pd.read_csv('../data/adri_data/avisos_detalle_zona.csv')
avisos_detalles_zona.head()

,nombre_zona,idzona_int
0,Gran Buenos Aires,7
1,Capital Federal,1
2,GBA Oeste,6
3,Buenos Aires (fuera de GBA),0
4,La Plata,8


In [149]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_zona, how='inner', on='nombre_zona')
avisos_detalles.drop('nombre_zona', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo,descripcion,contenido,idzona_int
0,1111556097,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249,8159438729029635585,7
1,1111949392,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002,7342170615448076428,7
2,1112145935,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304,4791830003540049985,7
3,1112146010,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648,2147483648,7
4,1112211475,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8,5853804619654291768,5925316576974522051,7


### Detalle Avisos (Nombre Ciudad)

In [150]:
avisos_detalles_ciudad = pd.DataFrame(avisos_detalles['ciudad'].unique(), columns=['ciudad'])
avisos_detalles_ciudad.fillna('', inplace=True)
# avisos_detalles_ciudad.index.name = 'idciudad'

avisos_detalles_ciudad['idciudad'] = avisos_detalles_ciudad['ciudad']
labelEncode(avisos_detalles_ciudad, 'idciudad')

avisos_detalles_ciudad.to_csv('../data/adri_data/avisos_detalle_ciudad.csv', sep=',', encoding='utf-8', index=False)
avisos_detalles_ciudad = pd.read_csv('../data/adri_data/avisos_detalle_ciudad.csv')
avisos_detalles_ciudad.head()

,ciudad,idciudad_int
0,NaN,0
1,Buenos Aires,5
2,Argentina,2
3,caba,49
4,paternal,50


In [151]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_ciudad, how='inner', on='ciudad')
avisos_detalles.drop('ciudad', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int
0,1111556097,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249,8159438729029635585,7,0
1,1111949392,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002,7342170615448076428,7,0
2,1112145935,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304,4791830003540049985,7,0
3,1112146010,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648,2147483648,7,0
4,1112211475,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8,5853804619654291768,5925316576974522051,7,0


### Detalle Avisos (Mapa Calle)

In [152]:
avisos_detalles_mapacalle = pd.DataFrame(avisos_detalles['mapacalle'].unique(), columns=['mapacalle'])
avisos_detalles_mapacalle.fillna('', inplace=True)
# avisos_detalles_mapacalle.index.name = 'idmapacalle'

avisos_detalles_mapacalle['idmapacalle'] = avisos_detalles_mapacalle['mapacalle']
labelEncode(avisos_detalles_mapacalle, 'idmapacalle')

avisos_detalles_mapacalle.to_csv('../data/adri_data/avisos_detalle_mapacalle.csv', sep=',', encoding='utf-8', index=False)
avisos_detalles_mapacalle = pd.read_csv('../data/adri_data/avisos_detalle_mapacalle.csv')
avisos_detalles_mapacalle.head()

,mapacalle,idmapacalle_int
0,NaN,0
1,Guemes 4747,430
2,Comodoro Rivadavia,339
3,Buenos Aires,276
4,Av Hipolito Yrigoyen 13500,86


In [153]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_mapacalle, how='inner', on='mapacalle')
avisos_detalles.drop('mapacalle', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int
0,1111556097,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0
1,1111949392,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002,7342170615448076428,7,0,0
2,1112145935,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304,4791830003540049985,7,0,0
3,1112146010,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648,2147483648,7,0,0
4,1112211475,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8,5853804619654291768,5925316576974522051,7,0,0


### Detalle Avisos (Tipo de Trabajo)

In [154]:
avisos_detalles_tipo_de_trabajo = pd.DataFrame(avisos_detalles['tipo_de_trabajo'].unique(), columns=['tipo_de_trabajo'])

avisos_detalles_tipo_de_trabajo.fillna('', inplace=True)
# avisos_detalles_tipo_de_trabajo.index.name = 'idtipo_de_trabajo'

avisos_detalles_tipo_de_trabajo['idtipo_de_trabajo'] = avisos_detalles_tipo_de_trabajo['tipo_de_trabajo']
labelEncode(avisos_detalles_tipo_de_trabajo, 'idtipo_de_trabajo')

avisos_detalles_tipo_de_trabajo.to_csv('../data/adri_data/avisos_detalle_tipo_de_trabajo.csv', sep=',', encoding='utf-8', index=False)
avisos_detalles_tipo_de_trabajo = pd.read_csv('../data/adri_data/avisos_detalle_tipo_de_trabajo.csv')
avisos_detalles_tipo_de_trabajo.head()

,tipo_de_trabajo,idtipo_de_trabajo_int
0,Full-time,1
1,Part-time,2
2,Temporario,8
3,Por Contrato,4
4,Teletrabajo,7


In [155]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_tipo_de_trabajo, how='inner', on='tipo_de_trabajo')
avisos_detalles.drop('tipo_de_trabajo', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,nivel_laboral,nombre_area,denominacion_empresa,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int
0,1111556097,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1
1,1111949392,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002,7342170615448076428,7,0,0,1
2,1112145935,Senior / Semi-Senior,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304,4791830003540049985,7,0,0,1
3,1112146010,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648,2147483648,7,0,0,1
4,1112211475,Senior / Semi-Senior,Comercial,RANDSTAD,-1,8,5853804619654291768,5925316576974522051,7,0,0,1


### Detalle Avisos (Nivel Laboral)

In [156]:
avisos_detalles_nivel_laboral = pd.DataFrame(avisos_detalles['nivel_laboral'].unique(), columns=['nivel_laboral'])

avisos_detalles_nivel_laboral.fillna('', inplace=True)
# avisos_detalles_nivel_laboral.index.name = 'idnivel_laboral'

avisos_detalles_nivel_laboral['idnivel_laboral'] = avisos_detalles_nivel_laboral['nivel_laboral']
labelEncode(avisos_detalles_nivel_laboral, 'idnivel_laboral')

avisos_detalles_nivel_laboral.to_csv('../data/adri_data/avisos_detalle_nivel_laboral.csv', sep=',', encoding='utf-8', index=False)
avisos_detalles_nivel_laboral = pd.read_csv('../data/adri_data/avisos_detalle_nivel_laboral.csv')
avisos_detalles_nivel_laboral.head()

,nivel_laboral,idnivel_laboral_int
0,Senior / Semi-Senior,5
1,Junior,3
2,Jefe / Supervisor / Responsable,2
3,Otro,4
4,Gerencia / Alta Gerencia / Dirección,1


In [157]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nivel_laboral, how='inner', on='nivel_laboral')
avisos_detalles.drop('nivel_laboral', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,nombre_area,denominacion_empresa,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int
0,1111556097,Desarrollo de Negocios,Sinteplast S.A.,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1,5
1,1111949392,Ingeniería Eléctrica y Electrónica,Roycan,1,2151677952,7413315525099389002,7342170615448076428,7,0,0,1,5
2,1112145935,Impuestos,Lighthouse Smart,-1,4684869787250065408,4719772409490604304,4791830003540049985,7,0,0,1,5
3,1112146010,Programación,Chevron Buenos Aires Shared Services Center,1,0,2147483648,2147483648,7,0,0,1,5
4,1112211475,Comercial,RANDSTAD,-1,8,5853804619654291768,5925316576974522051,7,0,0,1,5


### Detalle Avisos (Nombre Area)

In [162]:
avisos_detalles_nombre_area = pd.DataFrame(avisos_detalles['nombre_area'].unique(), columns=['nombre_area'])

avisos_detalles_nombre_area.fillna('', inplace=True)

avisos_detalles_nombre_area['idnombre_area'] = avisos_detalles_nombre_area['nombre_area']
labelEncode(avisos_detalles_nombre_area, 'idnombre_area')

avisos_detalles_nombre_area.to_csv('../data/adri_data/avisos_detalle_nombre_area.csv', sep=',', encoding='utf-8', index=False)
avisos_detalles_nombre_area = pd.read_csv('../data/adri_data/avisos_detalle_nombre_area.csv')
avisos_detalles_nombre_area.head()

,nombre_area,idnombre_area_int
0,Desarrollo de Negocios,50
1,Ingeniería Eléctrica y Electrónica,89
2,Impuestos,85
3,Programación,144
4,Comercial,29


In [163]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nombre_area, how='inner', on='nombre_area')
avisos_detalles.drop('nombre_area', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,denominacion_empresa,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int
0,1111556097,Sinteplast S.A.,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1,5,50
1,1112408768,Consultores de Empresas SRL,1,4611826755915743232,9025228747386659872,9097278288235072002,7,0,0,1,5,50
2,1112408781,Consultores de Empresas SRL,1,4900057132067454976,5490742658888377656,5491315436010080963,7,0,0,1,5,50
3,1112344682,Kaizen Recursos Humanos,1,4573968372072448,7652258028443976449,7579982640326689888,7,0,0,1,5,50
4,1112279443,Re/Max Net,-1,70368744185856,4936018171934015560,4936104672336806024,7,0,0,1,5,50


### Detalle Avisos (Denominacion Empresa)

In [164]:
avisos_detalles_denominacion_empresa = pd.DataFrame(avisos_detalles['denominacion_empresa'].unique(), columns=['denominacion_empresa'])

avisos_detalles_denominacion_empresa.fillna('', inplace=True)
# avisos_detalles_denominacion_empresa.index.name = 'iddenominacion_empresa'

avisos_detalles_denominacion_empresa['iddenominacion_empresa'] = avisos_detalles_denominacion_empresa['denominacion_empresa']
labelEncode(avisos_detalles_denominacion_empresa, 'iddenominacion_empresa')

avisos_detalles_denominacion_empresa.to_csv('../data/adri_data/avisos_detalle_denominacion_empresa.csv', sep=',', encoding='utf-8', index=False)
avisos_detalles_denominacion_empresa = pd.read_csv('../data/adri_data/avisos_detalle_denominacion_empresa.csv')
avisos_detalles_denominacion_empresa.head()

,denominacion_empresa,iddenominacion_empresa_int
0,Sinteplast S.A.,3680
1,Consultores de Empresas SRL,988
2,Kaizen Recursos Humanos,2298
3,Re/Max Net,3340
4,F & C Consultores y Asociados,1553


In [165]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_denominacion_empresa, how='inner', on='denominacion_empresa')
avisos_detalles.drop('denominacion_empresa', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,iddenominacion_empresa_int
0,1111556097,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1,5,50,3680
1,1112448447,1,576460769483300864,6228765605496356872,6157568899156279424,7,0,0,1,5,182,3680
2,1112394600,-1,1152921526081683456,7063905988445249617,6991840908279038473,1,0,920,1,5,34,3680
3,1111266888,1,18014398511579136,8106761422736344128,8107605375203478536,7,0,0,1,2,142,3680
4,1112426827,1,1125921381679104,8380078581765668944,8379514742535897097,7,0,0,1,5,114,3680


In [166]:
avisos_detalles.to_csv('../data/adri_data/avisos_detalles.csv', sep=',', encoding='utf-8', index=False)

# Agrego Features

In [36]:
postulaciones = pd.read_csv('../data/adri_data/postulaciones.csv', encoding='utf-8')
postulaciones.head()

,idaviso,idpostulante,year,month,day,dayofweek,hour
0,1112248724,NjlD,2018,1,19,4,7
1,1112286523,ZaO5,2018,1,24,2,15
2,1112272060,ZaO5,2018,1,24,2,15
3,1112288401,ZaO5,2018,1,26,4,8
4,1112300563,ZaO5,2018,1,30,1,13


In [37]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalles.csv', encoding='utf-8')
avisos_detalles.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,iddenominacion_empresa_int
0,1111556097,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1,5,50,3680
1,1112448447,1,576460769483300864,6228765605496356872,6157568899156279424,7,0,0,1,5,182,3680
2,1112394600,-1,1152921526081683456,7063905988445249617,6991840908279038473,1,0,920,1,5,34,3680
3,1111266888,1,18014398511579136,8106761422736344128,8107605375203478536,7,0,0,1,2,142,3680
4,1112426827,1,1125921381679104,8380078581765668944,8379514742535897097,7,0,0,1,5,114,3680


In [38]:
merged = pd.merge(postulaciones, avisos_detalles, how='inner', on=['idaviso'])
merged.head()

,idaviso,idpostulante,year,month,day,dayofweek,hour,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,iddenominacion_empresa_int
0,1112248724,NjlD,2018,1,19,4,7,-1,4611686018427404288,1328984380642172936,1257718159373501056,7,0,0,1,5,119,2469
1,1112248724,MV6eM8N,2018,1,30,1,23,-1,4611686018427404288,1328984380642172936,1257718159373501056,7,0,0,1,5,119,2469
2,1112248724,5LzQkN,2018,1,15,0,7,-1,4611686018427404288,1328984380642172936,1257718159373501056,7,0,0,1,5,119,2469
3,1112248724,8M2rOBz,2018,1,30,1,22,-1,4611686018427404288,1328984380642172936,1257718159373501056,7,0,0,1,5,119,2469
4,1112248724,Dr6V3Da,2018,1,23,1,15,-1,4611686018427404288,1328984380642172936,1257718159373501056,7,0,0,1,5,119,2469


In [39]:
def groupBy(merged, prop):
    preferida = pd.DataFrame(merged.groupby(['idpostulante', prop])[prop].count())
    preferida.rename({prop : 'count'}, inplace=True, axis=1)
    preferida = preferida.reset_index()
    preferida = preferida.sort_values('count', ascending = False).reset_index(drop=True)
    preferida = preferida.drop_duplicates('idpostulante')
    preferida = preferida.drop('count', axis=1)
    preferida.rename({prop : prop + '_pref'}, inplace=True, axis=1)
    return preferida

In [40]:
area_preferida = groupBy(merged, 'idnombre_area_int')
area_preferida.head()

,idpostulante,idnombre_area_int_pref
0,8MaQjP3,2
1,vVDd60Q,182
2,5Mwjak,2
3,epVRA4,142
4,KBzm0ZX,182


In [41]:
nivel_laboral_preferido = groupBy(merged, 'idnivel_laboral_int')
nivel_laboral_preferido.head()

,idpostulante,idnivel_laboral_int_pref
0,8MaQjP3,5
1,axmjOE,5
2,5Mwjak,5
3,KBzm0ZX,5
4,ZD2081P,5


In [42]:
tipo_de_trabajo_preferido = groupBy(merged, 'idtipo_de_trabajo_int')
tipo_de_trabajo_preferido.head()

,idpostulante,idtipo_de_trabajo_int_pref
0,8MaQjP3,1
1,axmjOE,1
2,KBzm0ZX,1
3,5Mwjak,1
4,ZD2081P,1


In [43]:
postulantes = pd.read_csv('../data/adri_data/postulantes.csv', encoding='utf-8')
postulantes.head()

,idpostulante,valor_educacion,educacion_int,nombre_int,estado_int,sexo,edad,birth_year,birth_month,birth_day
0,8rYD,16.0,4.0,1.0,1.0,-1,43,1974,12,11
1,e4VXj,16.0,4.0,1.0,1.0,-1,50,1968,4,25
2,a6GvZN,16.0,4.0,1.0,1.0,-1,48,1970,4,14
3,5264LM,16.0,4.0,1.0,1.0,-1,41,1976,7,30
4,5MlbrN,16.0,4.0,1.0,1.0,-1,40,1978,1,8


In [44]:
postulantes.shape

(504407, 10)

In [45]:
postulantes = pd.merge(postulantes, area_preferida, how='left', on='idpostulante')
postulantes = pd.merge(postulantes, nivel_laboral_preferido, how='left', on='idpostulante')
postulantes = pd.merge(postulantes, tipo_de_trabajo_preferido, how='left', on='idpostulante')
postulantes.fillna(0, inplace=True)
postulantes.head()

,idpostulante,valor_educacion,educacion_int,nombre_int,estado_int,sexo,edad,birth_year,birth_month,birth_day,idnombre_area_int_pref,idnivel_laboral_int_pref,idtipo_de_trabajo_int_pref
0,8rYD,16.0,4.0,1.0,1.0,-1,43,1974,12,11,0.0,0.0,0.0
1,e4VXj,16.0,4.0,1.0,1.0,-1,50,1968,4,25,142.0,5.0,1.0
2,a6GvZN,16.0,4.0,1.0,1.0,-1,48,1970,4,14,0.0,0.0,0.0
3,5264LM,16.0,4.0,1.0,1.0,-1,41,1976,7,30,0.0,0.0,0.0
4,5MlbrN,16.0,4.0,1.0,1.0,-1,40,1978,1,8,0.0,0.0,0.0


In [46]:
postulantes['idnombre_area_int_pref'] = postulantes['idnombre_area_int_pref'].astype('int')
postulantes['idnivel_laboral_int_pref'] = postulantes['idnivel_laboral_int_pref'].astype('int')
postulantes['idtipo_de_trabajo_int_pref'] = postulantes['idtipo_de_trabajo_int_pref'].astype('int')

postulantes['valor_educacion'] = postulantes['valor_educacion'].astype('int')

postulantes.head()

,idpostulante,valor_educacion,educacion_int,nombre_int,estado_int,sexo,edad,birth_year,birth_month,birth_day,idnombre_area_int_pref,idnivel_laboral_int_pref,idtipo_de_trabajo_int_pref
0,8rYD,16,4.0,1.0,1.0,-1,43,1974,12,11,0,0,0
1,e4VXj,16,4.0,1.0,1.0,-1,50,1968,4,25,142,5,1
2,a6GvZN,16,4.0,1.0,1.0,-1,48,1970,4,14,0,0,0
3,5264LM,16,4.0,1.0,1.0,-1,41,1976,7,30,0,0,0
4,5MlbrN,16,4.0,1.0,1.0,-1,40,1978,1,8,0,0,0


In [47]:
postulantes.shape

(504407, 13)

In [48]:
postulantes.head()

,idpostulante,valor_educacion,educacion_int,nombre_int,estado_int,sexo,edad,birth_year,birth_month,birth_day,idnombre_area_int_pref,idnivel_laboral_int_pref,idtipo_de_trabajo_int_pref
0,8rYD,16,4.0,1.0,1.0,-1,43,1974,12,11,0,0,0
1,e4VXj,16,4.0,1.0,1.0,-1,50,1968,4,25,142,5,1
2,a6GvZN,16,4.0,1.0,1.0,-1,48,1970,4,14,0,0,0
3,5264LM,16,4.0,1.0,1.0,-1,41,1976,7,30,0,0,0
4,5MlbrN,16,4.0,1.0,1.0,-1,40,1978,1,8,0,0,0


# Creo la lista de Ids numericos para los postulantes

In [49]:
# Convierto a numerico el id postulante
lb_make = LabelEncoder()
postulantes["idpostulante_int"] = lb_make.fit_transform(postulantes["idpostulante"])
postulantes[["idpostulante", "idpostulante_int"]].head()

,idpostulante,idpostulante_int
0,8rYD,115326
1,e4VXj,378335
2,a6GvZN,337123
3,5264LM,55046
4,5MlbrN,59562


In [208]:
postulantes.to_csv('../data/adri_data/postulantes2.csv', sep=',', encoding='utf-8', index=False)

# Agrupo Vistas, Postulaciones e Interacciones

In [64]:
postulantes = pd.read_csv('../data/adri_data/postulantes2.csv', sep=',', encoding='utf-8')
postulantes.head()

,idpostulante,valor_educacion,educacion_int,sexo,edad,idnombre_area_int_pref,idnivel_laboral_int_pref,idtipo_de_trabajo_int_pref,idpostulante_int
0,8rYD,16,4.0,2,43,0,0,0,115326
1,e4VXj,16,4.0,2,50,142,5,1,378335
2,a6GvZN,16,4.0,2,48,0,0,0,337123
3,5264LM,16,4.0,2,41,0,0,0,55046
4,5MlbrN,16,4.0,2,40,0,0,0,59562


In [51]:
postulaciones = pd.read_csv('../data/adri_data/postulaciones.csv', sep=',', encoding='utf-8')
postulaciones.head()

In [62]:
vistas = pd.read_csv('../data/adri_data/vistas.csv', sep=',', encoding='utf-8')
vistas.head()

,idaviso,idpostulante,year,month,day,dayofweek,hour
0,1112400775,kPjr2K0,2018,4,6,4,5
1,1112400775,kPjr2K0,2018,4,6,4,5
2,1112426664,GNOwZmb,2018,4,6,4,5
3,1112426664,GNOwZmb,2018,4,6,4,5
4,1112445738,lDLxajz,2018,4,6,4,5


In [60]:
postulaciones_por_postulante = postulaciones.groupby('idpostulante')['idaviso'].count().reset_index()
postulaciones_por_postulante.rename({'idaviso' : 'cantidad_postulaciones'}, axis=1, inplace=True)
postulaciones_por_postulante.head()

,idpostulante,cantidad_postulaciones
0,0z5Dmrd,2
1,0z5JW1r,7
2,0z5VvGv,50
3,0zB01pE,3
4,0zB026d,10


In [65]:
postulantes = pd.merge(postulantes, postulaciones_por_postulante, how='left')
postulantes.fillna(0, inplace=True)
postulantes.head()

,idpostulante,valor_educacion,educacion_int,sexo,edad,idnombre_area_int_pref,idnivel_laboral_int_pref,idtipo_de_trabajo_int_pref,idpostulante_int,cantidad_postulaciones
0,8rYD,16,4.0,2,43,0,0,0,115326,0.0
1,e4VXj,16,4.0,2,50,142,5,1,378335,1.0
2,a6GvZN,16,4.0,2,48,0,0,0,337123,0.0
3,5264LM,16,4.0,2,41,0,0,0,55046,0.0
4,5MlbrN,16,4.0,2,40,0,0,0,59562,0.0


In [63]:
vistas_por_postulante = vistas.groupby('idpostulante')['idaviso'].count().reset_index()
vistas_por_postulante.rename({'idaviso' : 'cantidad_vistas'}, axis=1, inplace=True)
vistas_por_postulante.head()

,idpostulante,cantidad_vistas
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [66]:
postulantes = pd.merge(postulantes, vistas_por_postulante, how='left')
postulantes.fillna(0, inplace=True)
postulantes.head()

,idpostulante,valor_educacion,educacion_int,sexo,edad,idnombre_area_int_pref,idnivel_laboral_int_pref,idtipo_de_trabajo_int_pref,idpostulante_int,cantidad_postulaciones,cantidad_vistas
0,8rYD,16,4.0,2,43,0,0,0,115326,0.0,5.0
1,e4VXj,16,4.0,2,50,142,5,1,378335,1.0,3.0
2,a6GvZN,16,4.0,2,48,0,0,0,337123,0.0,98.0
3,5264LM,16,4.0,2,41,0,0,0,55046,0.0,2.0
4,5MlbrN,16,4.0,2,40,0,0,0,59562,0.0,46.0


In [67]:
postulantes['cantidad_interacciones'] = postulantes['cantidad_postulaciones'] + postulantes['cantidad_vistas']

In [69]:
postulantes['educacion_int'] = postulantes['educacion_int'].astype(int)
postulantes['cantidad_postulaciones'] = postulantes['cantidad_postulaciones'].astype(int)
postulantes['cantidad_vistas'] = postulantes['cantidad_vistas'].astype(int)
postulantes['cantidad_interacciones'] = postulantes['cantidad_interacciones'].astype(int)

In [70]:
postulantes

,idpostulante,valor_educacion,educacion_int,sexo,edad,idnombre_area_int_pref,idnivel_laboral_int_pref,idtipo_de_trabajo_int_pref,idpostulante_int,cantidad_postulaciones,cantidad_vistas,cantidad_interacciones
0,8rYD,16,4,2,43,0,0,0,115326,0,5,5
1,e4VXj,16,4,2,50,142,5,1,378335,1,3,4
2,a6GvZN,16,4,2,48,0,0,0,337123,0,98,98
3,5264LM,16,4,2,41,0,0,0,55046,0,2,2
4,5MlbrN,16,4,2,40,0,0,0,59562,0,46,46
5,awQKED,16,4,1,36,29,5,1,356847,33,96,129
6,85BzMW,16,4,2,33,99,5,1,95931,1,3,4
7,6V1X2r,16,4,2,48,45,2,1,78515,1,4,5
8,5RjR9q,16,4,2,32,0,0,0,61094,0,13,13
9,6obdXl,16,4,1,44,119,4,1,81063,3,9,12


# Agrego cantidad de Vistas/Postulaciones por aviso

In [16]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalles.csv', encoding='utf-8')

In [4]:
vistas = pd.read_csv('../data/adri_data/vistas.csv', encoding='utf-8')
postulaciones = pd.read_csv('../data/adri_data/postulaciones.csv', encoding='utf-8')

In [17]:
vistas_por_aviso = vistas.groupby('idaviso')['idpostulante'].count().reset_index()
vistas_por_aviso.rename({'idpostulante' : 'cantidad_vistas'}, axis=1, inplace=True)
vistas_por_aviso.head()

,idaviso,cantidad_vistas
0,18,2
1,48375,1
2,169730,49
3,178417,1
4,189006,2


In [18]:
postulaciones_por_aviso = postulaciones.groupby('idaviso')['idpostulante'].count().reset_index()
postulaciones_por_aviso.rename({'idpostulante' : 'cantidad_postulaciones'}, axis=1, inplace=True)
postulaciones_por_aviso.head()

,idaviso,cantidad_postulaciones
0,8725750,137
1,11740890,13
2,12543760,1080
3,12812680,159
4,17903700,175


In [19]:
avisos_detalles = pd.merge(avisos_detalles, postulaciones_por_aviso, how='left', on='idaviso')
avisos_detalles.fillna(0, inplace=True)
avisos_detalles['cantidad_postulaciones'] = avisos_detalles['cantidad_postulaciones'].astype(int)
avisos_detalles.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,iddenominacion_empresa_int,cantidad_postulaciones
0,1111556097,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1,5,50,3680,227
1,1112448447,1,576460769483300864,6228765605496356872,6157568899156279424,7,0,0,1,5,182,3680,627
2,1112394600,-1,1152921526081683456,7063905988445249617,6991840908279038473,1,0,920,1,5,34,3680,564
3,1111266888,1,18014398511579136,8106761422736344128,8107605375203478536,7,0,0,1,2,142,3680,228
4,1112426827,1,1125921381679104,8380078581765668944,8379514742535897097,7,0,0,1,5,114,3680,451


In [20]:
avisos_detalles = pd.merge(avisos_detalles, vistas_por_aviso, how='left', on='idaviso')
avisos_detalles.fillna(0, inplace=True)
avisos_detalles['cantidad_vistas'] = avisos_detalles['cantidad_vistas'].astype(int)
avisos_detalles.head()

,idaviso,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,iddenominacion_empresa_int,cantidad_postulaciones,cantidad_vistas
0,1111556097,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1,5,50,3680,227,239.0
1,1112448447,1,576460769483300864,6228765605496356872,6157568899156279424,7,0,0,1,5,182,3680,627,2276.0
2,1112394600,-1,1152921526081683456,7063905988445249617,6991840908279038473,1,0,920,1,5,34,3680,564,494.0
3,1111266888,1,18014398511579136,8106761422736344128,8107605375203478536,7,0,0,1,2,142,3680,228,182.0
4,1112426827,1,1125921381679104,8380078581765668944,8379514742535897097,7,0,0,1,5,114,3680,451,1979.0


,idaviso,esta_online,titulo,descripcion,contenido,idzona_int,idciudad_int,idmapacalle_int,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,iddenominacion_empresa_int,cantidad_postulaciones,cantidad_vistas
0,1111556097,-1,1120986480640,8230659354068575249,8159438729029635585,7,0,0,1,5,50,3680,227,239
1,1112448447,1,576460769483300864,6228765605496356872,6157568899156279424,7,0,0,1,5,182,3680,627,2276
2,1112394600,-1,1152921526081683456,7063905988445249617,6991840908279038473,1,0,920,1,5,34,3680,564,494
3,1111266888,1,18014398511579136,8106761422736344128,8107605375203478536,7,0,0,1,2,142,3680,228,182
4,1112426827,1,1125921381679104,8380078581765668944,8379514742535897097,7,0,0,1,5,114,3680,451,1979
5,1112477605,-1,4683743633940152320,8230659354068575249,8159438729046412801,7,0,0,1,5,119,3680,0,326
6,1111628849,-1,4611686039902224384,8230615912445796373,8159401446039684609,1,0,914,1,5,45,3680,0,2
7,1112408768,1,4611826755915743232,9025228747386659872,9097278288235072002,7,0,0,1,5,50,988,40,62
8,1112408781,1,4900057132067454976,5490742658888377656,5491315436010080963,7,0,0,1,5,50,988,105,185
9,1112495854,-1,65536,6170004126571692192,6242130929264361490,7,0,0,1,5,50,988,0,444


In [ ]:
avisos_detalles = pd.read_csv('../data/adri_data/avisos_detalles_2.csv', encoding='utf-8')